### N-Gram Language Model

In [1]:
import pandas as pd
import ngram_utils as utils 

In [2]:
# constants 
NGRAM = 4
SENTENCE_BEGIN = "<s>"
SENTENCE_END = "</s>"

In [3]:
# read in cleaned data
song_df = pd.read_csv('clean_data.csv')
print(song_df.shape)
song_df.head(5)

(171855, 5)


,artist,song_name,lyrics,language,genres
0,ivete sangalo,Careless Whisper,I feel so unsure\nAs I take your hand and lead...,en,"['Pop', ' Axé', ' Romântico']"
1,ivete sangalo,Could You Be Loved / Citação Musical do Rap: S...,"Don't let them fool, ya\nOr even try to school...",en,"['Pop', ' Axé', ' Romântico']"
2,ivete sangalo,Cruisin' (Part. Saulo),"Baby, let's cruise, away from here\nDon't be c...",en,"['Pop', ' Axé', ' Romântico']"
3,ivete sangalo,Easy,"Know it sounds funny\nBut, I just can't stand ...",en,"['Pop', ' Axé', ' Romântico']"
4,ivete sangalo,For Your Babies (The Voice cover),You've got that look again\nThe one I hoped I ...,en,"['Pop', ' Axé', ' Romântico']"


#### Using NLTK Model with Kneser-Ney Smoothing 

In [4]:
power_pop_model = utils.create_ngram_kneser_ney_model(song_df, "Power-Pop", NGRAM)

In [5]:
for _ in range(10):
    sentence = list(power_pop_model.generate(20, "<s> <s>".split())) 
    sentence = " ".join(sentence)
    sentence = sentence.replace('</s>', '').strip()
    print(sentence)

power super super power
things might start improving
beth mackenzing sees one just up ahead
cars on the highway, up in the mirror is her only friend.
bible-bashing me in the eye?
freeloader freddy got a cold chevy nova
heart in my mouth; pulse in my head
(pre-chorus)
playing solitaire
freed the daytime with indifference


#### Using Own N-Gram Language Model with Laplace Smoothing

In [6]:
blues_model = utils.create_ngram_laplace_model(song_df, "Blues", NGRAM)

In [8]:
lyrics = blues_model.generate(10)
for lyric in lyrics:
    lyric = ' '.join(lyric)
    lyric = lyric.replace('<s>', '').replace('</s>', '').strip()
    print(lyric)

that nobody steals her heart away
<UNK> pledge my love to you is like a carousel you aim for heaven
coming 'round the bend.
la moora, la moora, soft as a sigh,
tumbling, in a world that has burned.
when i'm sad she comes to me
gets a big red nose
but when a love falls a heart must break,
don't give me a cool drink of water before i die
if all of my time
